In [1]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  scikit-image


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ········


In [3]:
from langchain.chat_models import init_chat_model


llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature=0.0) 

## Conversation Buffer Memory

In [5]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

/tmp/ipykernel_1363/1448044083.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [6]:
memory.save_context(
    {"input": "Hi, my names in James"},
    {"output": "Hey James, what's up? I'm an AI model called Zeta."}
)
memory.save_context(
    {"input": "I am researching different types of conversational memory."},
    {"output": "That's interesting, what are some examples?"}
)
memory.save_context(
    {"input": "I am researching ConversationBufferMemory and ConversationBufferWindowMemory"},
    {"output": "That's interesting, what's thed difference"}
)
memory.save_context(
    {"input": "Buffer memory just store the enitre converation, right?"},
    {"output": "Thats makes sense, what about ConversationBufferWindowMemory"}
)
memory.save_context(
    {"input": "Buffer window memory stores last k messages, dropping the rest."},
    {"output": "Very cool!"}
)


In [7]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  HumanMessage(conte

In [9]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)


/tmp/ipykernel_1363/2943370833.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [10]:
chain.invoke({"input": "What is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}

{'input': 'What is my name again?',
 'history': [HumanMessage(content='Hi, my names in James', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey James, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching different types of conversational memory.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am researching ConversationBufferMemory and ConversationBufferWindowMemory', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's thed difference", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just store the enitre converation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Thats makes sense, what about ConversationBufferWindowMemory', additional_kwargs={}, respons

## ConversationBufferMemory with RunnableWithMessageHistory

In [14]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}")   
])


In [43]:
pipeline = (
    { "query": lambda x: x["query"],
    "history": lambda x: x["history"]}
    | prompt_template
    | llm
)

In [44]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [45]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [46]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is James"},
    config={"session_id": "id_123"}
)

AIMessage(content="Hi James, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--adcbe9ea-bde0-480c-9a7b-1d5fd32b6d21-0', usage_metadata={'input_tokens': 14, 'output_tokens': 19, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

In [47]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

AIMessage(content='Your name is James.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ca735b9c-0f9a-48f8-a11f-99112cbba5f6-0', usage_metadata={'input_tokens': 38, 'output_tokens': 6, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}})